In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [ ]:
from read_data import split_dataset_with_ratio

class BatchDataset:
    def __init__(self,ratio,batch_size):
        data = split_dataset_with_ratio(ratio)
        self.data_train = data[0]
        self.labels_train = data[1]
        self.data_test = data[2]
        self.labels_test = data[3]
        self.counter = 0
        self.data_present = True
        self.batch_size = batch_size

    def get_next(self):
        if self.counter+self.batch_size<len(self.data_train):
            return (self.data_train[self.counter:self.counter+self.batch_size],self.labels_train[self.counter:self.counter+self.batch_size])
            self.counter += self.batch_size
        else:
            self.data_present = False
            return (self.data_train[self.counter:],self.labels_train[self.counter:])

    def get_test_data(self):
        return (self.data_test,self.labels_test)

    def get_details(self):
        print len(self.data_train), len(self.labels_train), len(self.data_test), len(self.labels_test)

In [ ]:
class Bais:
    def __init__(self,size):
        self.biases = tf.Variable(tf.constant(0.04, shape=[size]))

In [ ]:
class CNN:
    def __init__(self, input, num_of_input_channels, conv_filter_size, num_of_filters):
        self.weights = Weight(shape= [conv_filter_size, conv_filter_size, num_of_input_channels, num_of_filters])
        self.baises = Bais(size=num_of_filters)


        self.cnn = tf.nn.conv2d(input= input, filter= self.weights.weights, strides= [1, 1, 1, 1], padding= 'SAME')

        self.cnn += self.baises.biases

        self.cnn = tf.nn.max_pool(value= self.cnn, ksize= [1, 2, 2, 1], strides= [1, 2, 2, 1], padding= 'SAME')

        self.cnn = tf.nn.relu(self.cnn)

In [ ]:
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
    return layer

In [ ]:
def create_fully_connected_layer(input, no_of_inputs, no_of_outputs):
    weights = Weight(shape=[no_of_inputs, no_of_outputs])
    biases = Bais(no_of_outputs)
    layer = tf.matmul(input, weights.weights) + biases.biases
    return layer

In [ ]:
def one_hot_encode(labels):
    # if it is 'label1' it will be represented as [1, 0, 0] in one_hot_encoder. At a time only one output is hot
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[range(n_labels),labels] = 1
    one_hot_encode.astype(float)
    return one_hot_encode

def split_dataset_with_ratio(ratio):
    data = []
    labels = []
    flower_data_folder = "/home/sekhar/EXTRAS/flowers-recognition/flowers_reshaped/"
    for each_flower_image in glob.glob(flower_data_folder+"*.jpg"):
        image_data = cv2.imread(each_flower_image)
        image_data = image_data.astype(float)
        image_label = each_flower_image.split('/')[-1].split('_')[0]
        data.append(image_data)
        labels.append(image_label)
        print image_label
        print image_data

    data, labels = shuffle(data, labels, random_state=1)
    labelEncoder = LabelEncoder()
    labelEncoder.fit(labels)
    labels = labelEncoder.transform(labels)
    labels = one_hot_encode(labels=labels)
    data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=ratio, random_state=42)
    return data_train, labels_train, data_test, labels_test

In [ ]:
class Weight:
    def __init__(self,shape):
        self.weights = tf.Variable(tf.truncated_normal(shape = shape, stddev=0.04))

In [1]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
FILTERS = 3
NO_OF_FLOWERS = 5
EPOCHS = 50


input = tf.placeholder(tf.float32, shape=[None, IMAGE_HEIGHT,IMAGE_WIDTH,FILTERS])
output_true = tf.placeholder(tf.float32, shape=[None, NO_OF_FLOWERS])



layer1 = CNN(input=input, num_of_input_channels=3, conv_filter_size=8, num_of_filters=3).cnn
layer2 = CNN(input=layer1, num_of_input_channels=3, conv_filter_size=8, num_of_filters=3).cnn
layer3 = CNN(input=layer2, num_of_input_channels=3, conv_filter_size= 8, num_of_filters= 3).cnn
layer4 = CNN(input=layer3, num_of_input_channels=3, conv_filter_size= 8, num_of_filters= 3).cnn

layer_flat = flatten(layer4)

layer_fully_connected = create_fully_connected_layer(input=layer_flat,no_of_inputs=layer_flat.get_shape()[1:4].num_elements(),no_of_outputs=5)

# fileWriter = tf.summary.FileWriter(".",sess.graph)
print(layer_fully_connected)

cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= layer_fully_connected, labels= output_true))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost_function)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batchDataset = BatchDataset(0.30,4)
print "dataset content details:"
batchDataset.get_details()

# for each_epoch in range(EPOCHS):
#     sess.run(optimizer, feed_dict={input: data_train, output_true: labels_train})
#     cost = sess.run(cost_function,feed_dict={input:data_train, output_true:labels_train})
#     correct_predictions = tf.equal(tf.argmax(layer_fully_connected,1),tf.argmax(output_true,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
#     pred_outputs = sess.run(layer_fully_connected,{input:data_test})
#     mean_square_error = tf.reduce_mean(tf.square(pred_outputs - labels_test))
#     mean_square_error_value = sess.run(mean_square_error)
#     accuracy_value = sess.run(accuracy, {input: data_test, output_true: labels_test})
#     print "epoch number : "+ str(each_epoch) + " - cost : "+ str(cost)+ " - mse : "+ str(mean_square_error_value)+" - accuracy : "+str(accuracy_value)


for each_epoch in range(EPOCHS):
    batchDataset.data_present = True
    batchDataset.counter = 0
    data_test, labels_test = batchDataset.get_test_data()
    while(batchDataset.data_present):
        data_train, labels_train = batchDataset.get_next()
        print (data_train,labels_train)
        sess.run(optimizer, feed_dict={input: data_train, output_true: labels_train})
        cost = sess.run(cost_function,feed_dict={input:data_train, output_true:labels_train})
        correct_predictions = tf.equal(tf.argmax(layer_fully_connected,1),tf.argmax(output_true,1))
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
        pred_outputs = sess.run(layer_fully_connected,{input:data_test})
        mean_square_error = tf.reduce_mean(tf.square(pred_outputs - labels_test))
        mean_square_error_value = sess.run(mean_square_error)
        accuracy_value = sess.run(accuracy, {input: data_test, output_true: labels_test})
        print "epoch number : "+ str(each_epoch) + " - cost : "+ str(cost)+" - mse : "+str(mean_square_error_value)+" - accuracy : "+str(accuracy_value)


saver = tf.train.Saver()
save_path = saver.save(sess, "/home/sekhar/EXTRAS/tensorflow-basics/Flower-Dataset")
sess.close()

NameError: name 'tf' is not defined